In [51]:
import pandas as pd
import sqlite3

In [52]:
import os

try:
    SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # __file__ doesn't exist (e.g., in Jupyter)
    SCRIPT_DIR = os.getcwd()  # use current working directory

LOCAL_DB_PATH = os.path.join(
    os.path.dirname(os.path.dirname(os.path.dirname(SCRIPT_DIR))),
    "data",
    "test_database.db"
)

DB_PATH = os.getenv("DATABASE_PATH", LOCAL_DB_PATH)
print(f"Using database path: {DB_PATH}")


Using database path: c:\Users\benwa\OneDrive\Desktop\side_projects\FlexLog\data\test_database.db


In [53]:
def get_input_features(user_id: int):
  with sqlite3.connect(DB_PATH) as conn:
    cursor = conn.cursor()
    cursor.execute("""
        SELECT 
            l.timestamp, l.first,
            w.name as workout_name,
            e.variant,
            e.machine_type,
            e.name as exercise_name,
            e.chest, e.back, e.legs, e.shoulders, e.biceps, e.triceps, e.misc_group,
            e.barbell, e.dumbbell, e.machine, e.cable, e.smith, e.misc_machine,
            e.isolation, e.compound
        FROM logs l
        JOIN exercises e ON l.exercise_id = e.id
        JOIN workouts w ON l.workout_id = w.id
        WHERE l.user_id = ?
        GROUP BY l.exercise_id, l.workout_id
        ORDER BY l.timestamp ASC
    """, (user_id,))
    rows = cursor.fetchall()
    return rows

 

In [54]:
rows = get_input_features(1)

column_names = [
    "timestamp", "first", "workout_name", 
    "variant", "machine_type", "exercise_name",
    "chest","back","legs","shoulders","biceps","triceps","misc_group",
    "barbell","dumbbell","machine","cable","smith","misc_machine",
    "isolation","compound"
]

df = pd.DataFrame(rows, columns=column_names)

df

,timestamp,first,workout_name,variant,machine_type,exercise_name,chest,back,legs,shoulders,...,triceps,misc_group,barbell,dumbbell,machine,cable,smith,misc_machine,isolation,compound
0,2026-01-06 18:00:00,1,chest shoulders trcieps,,dumbbell,skullcrusher,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
1,2026-01-06 18:20:00,0,chest shoulders trcieps,rope,,tricep pushdown,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
2,2026-01-06 18:50:00,0,chest shoulders trcieps,,machine,chest press,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,2026-01-06 19:10:00,0,chest shoulders trcieps,,barbell,bench press,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,2026-01-06 19:30:00,0,chest shoulders trcieps,single-arm,cable,lateral raise,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
5,2026-01-06 20:00:00,0,chest shoulders trcieps,,dumbbell,overhead press,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
6,2026-01-06 20:30:00,0,chest shoulders trcieps,,,cable rear delt fly,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
7,2026-01-08 18:00:00,1,back biceps,rope,cable,curl,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
8,2026-01-08 18:30:00,0,back biceps,,dumbbell,hammer curl,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
9,2026-01-08 18:50:00,0,back biceps,,,machine row,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1


In [55]:
df["exercise_name"] = df["variant"] + " " +  df["machine_type"] + " " + df["exercise_name"]

muscle_groups = ["chest", "back", "legs", "shoulders", "biceps", "triceps"]

for group in muscle_groups: 
  df[f"{group}_day"] = df["workout_name"].str.contains(group, case=False).astype(int)


In [56]:

lagged_cols = ["barbell", "dumbbell", "machine", "cable", "smith", "misc_machine", "misc_group", "isolation", "compound", "chest", "back", "legs", "shoulders", "biceps", "triceps"]

for col in lagged_cols:
    df[f"prev_{col}"] = (
        df[col]
        .shift(1)
        .fillna(0)
        .astype(int)
    )

    df.loc[df["first"] == 1, f"prev_{col}"] = 0

In [57]:
feature_cols = [col for col in df.columns if col.startswith("prev_") or "day" in col]

X = df[feature_cols].values
y_muscle = df[["chest", "back", "legs", "shoulders", "biceps", "triceps", "misc_group"]]
y_machine = df[["barbell", "dumbbell", "machine", "cable", "smith", "misc_machine"]]
y_type = df[["isolation", "compound"]]

In [58]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# ------------------------------------------------------------------
# 1. SINGLE train/test split using DataFrame indices
# ------------------------------------------------------------------

train_idx, test_idx = train_test_split(
    df.index,
    test_size=0.4,
    random_state=42
)

X_train = X[train_idx]
X_test  = X[test_idx]

y_muscle_train  = y_muscle.iloc[train_idx]
y_muscle_test   = y_muscle.iloc[test_idx]

y_machine_train = y_machine.iloc[train_idx]
y_machine_test  = y_machine.iloc[test_idx]

y_type_train    = y_type.iloc[train_idx]
y_type_test     = y_type.iloc[test_idx]

# Keep aligned DataFrame rows
test_df = df.loc[test_idx]

# ------------------------------------------------------------------
# 2. Train Ridge models
# ------------------------------------------------------------------

ridge_muscle  = Ridge(alpha=1.0).fit(X_train, y_muscle_train)
ridge_machine = Ridge(alpha=1.0).fit(X_train, y_machine_train)
ridge_type    = Ridge(alpha=1.0).fit(X_train, y_type_train)

# ------------------------------------------------------------------
# 3. Predict + argmax
# ------------------------------------------------------------------

y_muscle_pred_class  = ridge_muscle.predict(X_test).argmax(axis=1)
y_machine_pred_class = ridge_machine.predict(X_test).argmax(axis=1)
y_type_pred_class    = ridge_type.predict(X_test).argmax(axis=1)

y_muscle_true_class  = y_muscle_test.to_numpy().argmax(axis=1)
y_machine_true_class = y_machine_test.to_numpy().argmax(axis=1)
y_type_true_class    = y_type_test.to_numpy().argmax(axis=1)

# ------------------------------------------------------------------
# 4. (Optional) Metrics
# ------------------------------------------------------------------

# print("Muscle accuracy:", accuracy_score(y_muscle_true_class, y_muscle_pred_class))
# print("Machine accuracy:", accuracy_score(y_machine_true_class, y_machine_pred_class))
# print("Type accuracy:", accuracy_score(y_type_true_class, y_type_pred_class))

# ------------------------------------------------------------------
# 5. Label decoding
# ------------------------------------------------------------------

muscle_labels  = ["chest","back","legs","shoulders","biceps","triceps","misc_group"]
machine_labels = ["barbell","dumbbell","machine","cable","smith","misc_machine"]
type_labels    = ["isolation","compound"]

predicted_muscle  = [muscle_labels[i] for i in y_muscle_pred_class]
predicted_machine = [machine_labels[i] for i in y_machine_pred_class]
predicted_type    = [type_labels[i] for i in y_type_pred_class]

# ------------------------------------------------------------------
# 6. Decode prev_* features
# ------------------------------------------------------------------

def get_prev_from_onehot(row, labels, prefix):
    for label in labels:
        if row[f"{prefix}{label}"] == 1:
            return label
    return "none"

final_df = pd.DataFrame({
    "prev_muscle":  test_df.apply(lambda r: get_prev_from_onehot(r, muscle_labels, "prev_"), axis=1),
    "prev_machine": test_df.apply(lambda r: get_prev_from_onehot(r, machine_labels, "prev_"), axis=1),
    "prev_type":    test_df.apply(lambda r: get_prev_from_onehot(r, type_labels, "prev_"), axis=1),
    "workout_name": test_df["workout_name"],
    "next_group":   predicted_muscle,
    "next_machine": predicted_machine,
    "next_type":    predicted_type
})

final_df


,prev_muscle,prev_machine,prev_type,workout_name,next_group,next_machine,next_type
39,biceps,dumbbell,isolation,chest biceps shoulders,triceps,cable,isolation
25,shoulders,cable,isolation,chest shoulders trcieps,shoulders,dumbbell,compound
26,triceps,dumbbell,isolation,chest shoulders trcieps,triceps,cable,isolation
43,triceps,cable,isolation,back triceps,back,misc_machine,compound
35,none,none,none,chest biceps shoulders,biceps,cable,isolation
41,none,none,none,back triceps,back,cable,isolation
4,chest,barbell,compound,chest shoulders trcieps,triceps,cable,isolation
12,none,none,none,chest biceps shoulders,biceps,cable,isolation
8,biceps,cable,isolation,back biceps,back,machine,compound
3,chest,machine,compound,chest shoulders trcieps,triceps,cable,isolation
